In [72]:
# Keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD # 確率的勾配降下法

from keras.initializers import glorot_uniform, orthogonal, TruncatedNormal # 初期化
from keras.preprocessing.image import load_img, array_to_img, img_to_array # 画像の取り扱い
from keras.utils import np_utils, plot_model # ?

from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_resnet_v2 import InceptionResNetV2

from keras.layers import LSTM
from keras.layers.recurrent import GRU, SimpleRNN


import pandas as pd #行列計算
import numpy as np #行列計算
import math #数値計算
import itertools #順列・組み合わせ

import matplotlib.pyplot as plt #グラフ
import seaborn as sns

from datetime import datetime, timedelta
import mpl_finance as mpf
import talib as ta # テクニカル指標

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.ensemble import RandomForestRegressor, VotingClassifier

# GBDT
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
#from catboost import CatBoostClassifier, Pool

# フォント
import matplotlib.font_manager as fm
from matplotlib import rcParams

from PIL import Image, ImageFilter
import glob
import re
import uuid

In [73]:
# 余白を追加
def add_margin(pil_img, top, right, bottom, left, color):
    width, height = pil_img.size
    new_width = width + right + left
    new_height = height + top + bottom
    result = Image.new(pil_img.mode, (new_width, new_height), color)
    result.paste(pil_img, (left, top))
    return result

In [74]:
# 余白を追加して正方形に変形
def expand2square(pil_img, background_color):
    width, height = pil_img.size
    if width == height:
        return pil_img
    elif width > height:
        result = Image.new(pil_img.mode, (width, width), background_color)
        result.paste(pil_img, (0, (width - height) // 2))
        return result
    else:
        result = Image.new(pil_img.mode, (height, height), background_color)
        result.paste(pil_img, ((height - width) // 2, 0))
        return result

In [75]:
img_size = 64# 256

## ドール画像の回帰分析

## 画像の読み込み

In [77]:
X = []
y = []

img_path_list = glob.glob('image/ddh-01_aucfree_custom_head_only/*')

for path in img_path_list:
    print(path)
    img = Image.open(path)
    img = expand2square(img, (255, 255, 255))
    img = img.resize((img_size, img_size))
    
    
    X.append(np.array(img))
    y.append(0)
    
    """
    img = img.transpose(Image.FLIP_LEFT_RIGHT) # 左右反転
    X.append(np.array(img))
    y.append(0)
    img = img.transpose(Image.FLIP_TOP_BOTTOM) # 上下反転
    X.append(np.array(img))
    y.append(0)
    img = img.transpose(Image.ROTATE_90) # 90度回転
    X.append(np.array(img))
    y.append(0)
    img = img.transpose(Image.ROTATE_180) # 180度回転
    X.append(np.array(img))
    y.append(0)
    img = img.transpose(Image.ROTATE_270) # 270度回転
    X.append(np.array(img))
    y.append(0)
    """

img_path_list = glob.glob('image/ddh-01_aucfree_not_custom_head_only/*')
for path in img_path_list:
    print(path)
    img = Image.open(path)
    img = expand2square(img, (255, 255, 255))
    img = img.resize((img_size, img_size))
    
    X.append(np.array(img)) # 訓練データのリストに追加
    y.append(1)
    
    #img = img.transpose(Image.FLIP_LEFT_RIGHT) # 左右反転
    #X.append(np.array(img))
    #y.append(1)
    #img = img.transpose(Image.FLIP_TOP_BOTTOM) # 上下反転
    #X.append(np.array(img))
    #y.append(1)
    #img = img.transpose(Image.ROTATE_90) # 90度回転
    #X.append(np.array(img))
    #y.append(1)
    #img = img.transpose(Image.ROTATE_180) # 180度回転
    #X.append(np.array(img))
    #y.append(1)
    #img = img.transpose(Image.ROTATE_270) # 270度回転
    #X.append(np.array(img))
    #y.append(1)


image/ddh-01_aucfree_custom_head_only\00001000_bff6ee59-407e-4553-bfbb-264782fdbb4c.jpeg
image/ddh-01_aucfree_custom_head_only\00001200_fd06596e-0569-4fe5-a1f9-8969dad054bd.jpeg
image/ddh-01_aucfree_custom_head_only\00002000_73b24590-c7c3-480f-a7fa-e2b0c36d73d2.jpeg
image/ddh-01_aucfree_custom_head_only\00002000_a1cea8ae-8055-4064-86ab-bde78e1a3e0d.jpeg
image/ddh-01_aucfree_custom_head_only\00002000_d62c7d0b-a133-4601-9fbe-cab22953046f.jpeg
image/ddh-01_aucfree_custom_head_only\00002000_ee340ea2-8e3f-435f-9260-5e52aa968061.jpeg
image/ddh-01_aucfree_custom_head_only\00002600_e97c873b-517f-45ef-b931-7dfcce04fb9a.jpeg
image/ddh-01_aucfree_custom_head_only\00002800_01ab0b7c-f30b-49d4-be07-fcc58f887a01.jpeg
image/ddh-01_aucfree_custom_head_only\00002800_60bb5355-41a5-44d6-a71d-c0fb0d37a1ec.jpeg
image/ddh-01_aucfree_custom_head_only\00002800_d4c5b7e8-9277-4ef1-83f1-17a20d85939d.jpeg
image/ddh-01_aucfree_custom_head_only\00003000_10a36d23-f53c-4bb2-b811-cc08ba636377.jpeg
image/ddh-01_aucfree_

image/ddh-01_aucfree_custom_head_only\00005000_80f5f05e-1cc3-408c-beac-04bc7b8c962b.jpeg
image/ddh-01_aucfree_custom_head_only\00005000_82542ddf-d1ea-4981-a24b-62258f7bab4e.jpeg
image/ddh-01_aucfree_custom_head_only\00005000_85d9fe78-4db1-40a0-8210-56bdbcbee9e4.jpeg
image/ddh-01_aucfree_custom_head_only\00005000_8e2501a1-0900-43aa-8c1c-1ea1ebf62c9e.jpeg
image/ddh-01_aucfree_custom_head_only\00005000_910d819a-f2ad-4583-bab9-c3a98c2971c7.jpeg
image/ddh-01_aucfree_custom_head_only\00005000_9123be25-178c-40ed-89c8-b059ceeee2e0.jpeg
image/ddh-01_aucfree_custom_head_only\00005000_91a37d4f-56e4-4554-95d0-a0d10e99c545.jpeg
image/ddh-01_aucfree_custom_head_only\00005000_92263a3a-e1f0-415c-954d-9b83eb06422e.jpeg
image/ddh-01_aucfree_custom_head_only\00005000_925d12d3-4da0-43e5-b099-25644c189769.jpeg
image/ddh-01_aucfree_custom_head_only\00005000_9b614b87-87c2-416b-a0bf-ac1e436d548d.jpeg
image/ddh-01_aucfree_custom_head_only\00005000_9d450f0a-64a0-4bfe-be1d-8201142b6b9d.jpeg
image/ddh-01_aucfree_

image/ddh-01_aucfree_custom_head_only\00005800_8c3bc55b-92ce-468b-84bc-f0044fe25d4a.jpeg
image/ddh-01_aucfree_custom_head_only\00005800_babb28c1-d414-4081-93a9-2b4fa3ef4991.jpeg
image/ddh-01_aucfree_custom_head_only\00005800_f76e7d77-2e21-4433-baea-2d6d2339ba08.jpeg
image/ddh-01_aucfree_custom_head_only\00005950_c81e67ec-b543-4604-abaf-941dff5aea17.jpeg
image/ddh-01_aucfree_custom_head_only\00006000_011e1e29-cb5e-42a1-9aff-eca5093596dd.jpeg
image/ddh-01_aucfree_custom_head_only\00006000_0446ec1e-f6ec-4f7c-9799-57937cc69312.jpeg
image/ddh-01_aucfree_custom_head_only\00006000_05967cae-98c8-4843-a09d-fc17b1743ec9.jpeg
image/ddh-01_aucfree_custom_head_only\00006000_0a99ea8a-ff8e-44b2-a577-7fba0b89715f.jpeg
image/ddh-01_aucfree_custom_head_only\00006000_0e0b04c5-6dc5-4e8e-8e01-0c9b7f04d743.jpeg
image/ddh-01_aucfree_custom_head_only\00006000_1846351b-13c8-4f0b-bfe8-18f8b4f64750.jpeg
image/ddh-01_aucfree_custom_head_only\00006000_1928327f-e2dd-4025-a861-6ad3ba29ef64.jpeg
image/ddh-01_aucfree_

image/ddh-01_aucfree_custom_head_only\00006700_a473a54a-1a5f-4e64-a8bd-38471d63600c.jpeg
image/ddh-01_aucfree_custom_head_only\00006740_b14a439a-4d6a-4b35-86b5-96057ca2144a.jpeg
image/ddh-01_aucfree_custom_head_only\00006750_1d12ed8c-5003-411b-a845-3defb21279eb.jpeg
image/ddh-01_aucfree_custom_head_only\00006750_2cabc309-fea8-4674-9b78-5294a3efc5c7.jpeg
image/ddh-01_aucfree_custom_head_only\00006750_3933aa63-e05b-432f-9b89-10c2e96d80b5.jpeg
image/ddh-01_aucfree_custom_head_only\00006750_413c90a8-ec72-47e3-b798-3fe20609e2d7.jpeg
image/ddh-01_aucfree_custom_head_only\00006750_46f2994c-2a80-41fb-974c-453e8533f413.jpeg
image/ddh-01_aucfree_custom_head_only\00006750_4c44297a-a62c-4157-82de-c29fd697b879.jpeg
image/ddh-01_aucfree_custom_head_only\00006750_4ea2df1f-3b31-4ebd-89bc-aad5994d6b3f.jpeg
image/ddh-01_aucfree_custom_head_only\00006750_5181e521-392b-4c7d-83fd-4317014f4fd8.jpeg
image/ddh-01_aucfree_custom_head_only\00006750_5243b045-222b-4946-bcce-35c391c6afba.jpeg
image/ddh-01_aucfree_

image/ddh-01_aucfree_custom_head_only\00007250_860ea941-1ee2-49e3-b08e-68c3939d9f63.jpeg
image/ddh-01_aucfree_custom_head_only\00007250_9ce9288f-6e1e-4181-b29c-478fbd732d99.jpeg
image/ddh-01_aucfree_custom_head_only\00007250_a6bb7370-e79f-4708-9a41-ed07faea01f2.jpeg
image/ddh-01_aucfree_custom_head_only\00007250_a7f4a6ab-2f8c-404d-b85e-d0e55b1625a7.jpeg
image/ddh-01_aucfree_custom_head_only\00007250_aaf6d98e-86e1-44c1-84e3-77bff7de55a6.jpeg
image/ddh-01_aucfree_custom_head_only\00007250_abb79b79-b4b7-46dd-8d6b-b3239b477860.jpeg
image/ddh-01_aucfree_custom_head_only\00007250_b9648d60-9706-4efe-ba65-819fc4ee42e9.jpeg
image/ddh-01_aucfree_custom_head_only\00007250_ebe30288-49ad-4fa8-af6d-e404f3de3a35.jpeg
image/ddh-01_aucfree_custom_head_only\00007250_ed81f9de-7605-4369-8f07-17496bd271e9.jpeg
image/ddh-01_aucfree_custom_head_only\00007340_f0bc4602-7d9f-45c0-8aa0-167b84e1a849.jpeg
image/ddh-01_aucfree_custom_head_only\00007350_5a16e56b-46ce-4988-b900-7630dec7f6c1.jpeg
image/ddh-01_aucfree_

...

image/ddh-01_aucfree_not_custom_head_only\00004500_a2b981db-d2d5-4b16-9a1f-a572401c8e10.jpeg
image/ddh-01_aucfree_not_custom_head_only\00004500_dd14457e-f613-4e88-9a1b-5447076532d6.jpeg
image/ddh-01_aucfree_not_custom_head_only\00004600_291685cc-52c4-42ae-9055-354a6880a5f5.jpeg
image/ddh-01_aucfree_not_custom_head_only\00004600_3ae027e5-dbd9-4998-92aa-80572d98cdc9.jpeg
image/ddh-01_aucfree_not_custom_head_only\00004600_4680c146-a448-488d-bb78-f820819c160a.jpeg
image/ddh-01_aucfree_not_custom_head_only\00004600_6ea54906-5394-4b61-9926-5c56e8740e65.jpeg
image/ddh-01_aucfree_not_custom_head_only\00004600_7f8e5c40-c695-4570-9857-379b90a92864.jpeg
image/ddh-01_aucfree_not_custom_head_only\00004600_b41f0b40-a659-449c-81cd-288bb57d6b52.jpeg
image/ddh-01_aucfree_not_custom_head_only\00004600_de006a17-d979-40dd-ae1d-367ad47c875d.jpeg
image/ddh-01_aucfree_not_custom_head_only\00004600_f0c4df97-14c4-4317-b7c2-388551c4b064.jpeg
image/ddh-01_aucfree_not_custom_head_only\00004800_077346cb-b244-4561-

image/ddh-01_aucfree_not_custom_head_only\00008000_753fc393-42e1-4114-b760-6bc4395f3984.jpeg
image/ddh-01_aucfree_not_custom_head_only\00008000_85bbe6f3-74b6-4e56-97bc-db4ab302138a.jpeg
image/ddh-01_aucfree_not_custom_head_only\00008000_94e9d470-2556-4360-ba15-982562bf462b.jpeg
image/ddh-01_aucfree_not_custom_head_only\00008000_b8160755-c28c-47f1-98d4-76acf1d331e4.jpeg
image/ddh-01_aucfree_not_custom_head_only\00008000_d9ace413-f5bd-4929-a44a-e11fb88acc8e.jpeg
image/ddh-01_aucfree_not_custom_head_only\00008250_36eee55c-f408-4720-863d-52c7747a515e.jpeg
image/ddh-01_aucfree_not_custom_head_only\00008250_a284a5e8-8b75-4529-8512-ca335a06f622.jpeg
image/ddh-01_aucfree_not_custom_head_only\00008250_a714fbba-c0fb-4d19-9f68-095db893fa66.jpeg
image/ddh-01_aucfree_not_custom_head_only\00008250_eaf906c4-12a9-4ee3-836a-95063033ab9b.jpeg
image/ddh-01_aucfree_not_custom_head_only\00008300_78d3ff7b-a047-4eb4-971d-06c877ad7fcc.jpeg
image/ddh-01_aucfree_not_custom_head_only\00008500_5ce81305-a2b4-4cf3-

image/ddh-01_aucfree_not_custom_head_only\00014500_2d86329b-ea6f-489b-a92a-d20c280fc470.jpeg
image/ddh-01_aucfree_not_custom_head_only\00014500_8b04d1b4-2ddf-465b-8100-9ee560db984b.jpeg
image/ddh-01_aucfree_not_custom_head_only\00014500_c078563a-b99e-4396-8a48-5e7091c12839.jpeg
image/ddh-01_aucfree_not_custom_head_only\00015000_0244646b-bc5b-4a2f-88aa-f0c647d8b786.jpeg
image/ddh-01_aucfree_not_custom_head_only\00015000_1cc3e5da-1b08-4771-b0ea-0fb31f657d4a.jpeg
image/ddh-01_aucfree_not_custom_head_only\00015000_4da19d61-8ea3-445c-9b8a-d827bd6bc419.jpeg
image/ddh-01_aucfree_not_custom_head_only\00015000_989b4399-5e5e-4ddc-a364-eb79ab0b8aa6.jpeg
image/ddh-01_aucfree_not_custom_head_only\00015000_c917d80d-16b1-4071-be96-a99df909a03a.jpeg
image/ddh-01_aucfree_not_custom_head_only\00015000_d3044873-cfa6-4e0f-8567-64367d98ac5a.jpeg
image/ddh-01_aucfree_not_custom_head_only\00015000_d5903e2d-e979-4402-b341-cde123d7c9b2.jpeg
image/ddh-01_aucfree_not_custom_head_only\00015000_e86c8bd6-418b-41d7-

image/ddh-01_aucfree_not_custom_head_only\00027000_ef43071a-ada0-493a-8da7-21240fa0299c.jpeg
image/ddh-01_aucfree_not_custom_head_only\00027100_b9da9d7a-cce0-4fe9-956f-fa776f31f40a.jpeg
image/ddh-01_aucfree_not_custom_head_only\00027450_c6494aeb-b40e-42d3-a482-1b1eb388a8b8.jpeg
image/ddh-01_aucfree_not_custom_head_only\00027500_36ae5b2d-ea0c-4559-87e3-d8914a88d001.jpeg
image/ddh-01_aucfree_not_custom_head_only\00027500_4f27aa9d-0768-414f-9ae9-198ffdd76074.jpeg
image/ddh-01_aucfree_not_custom_head_only\00028000_466f093b-d5e3-4fed-8cf3-4716cda87dcd.jpeg
image/ddh-01_aucfree_not_custom_head_only\00028500_1a70ebf7-fc6d-4eda-91a5-9548f6771adc.jpeg
image/ddh-01_aucfree_not_custom_head_only\00028500_5d8e3a72-e15c-4468-abed-f211366a61ec.jpeg
image/ddh-01_aucfree_not_custom_head_only\00028500_7d779f4c-dbca-496b-874c-b0b178bfb22f.jpeg
image/ddh-01_aucfree_not_custom_head_only\00028500_8d439f9b-b7fa-4ac7-bc96-610b86d7e942.jpeg
image/ddh-01_aucfree_not_custom_head_only\00028500_96b815a2-0ab4-4c51-

image/ddh-01_aucfree_not_custom_head_only\00301000_623d9743-ad15-4fa0-a273-daee4589d5ac.jpeg
image/ddh-01_aucfree_not_custom_head_only\00311000_9d8bf444-f8fb-4199-8d65-de910435cb24.jpeg
image/ddh-01_aucfree_not_custom_head_only\00350000_0974e3b5-d1a8-4a63-b978-d18a96d08f9e.jpeg
image/ddh-01_aucfree_not_custom_head_only\00351000_fd80cb8e-bda8-40ad-85eb-988662e5eab8.jpeg
image/ddh-01_aucfree_not_custom_head_only\00679000_fa68cc37-dac6-46aa-87fa-f3434dac7ce3.jpeg


In [78]:
# numpy配列に変換
X = np.array(X)
y = np.array(y)
y = np.eye(2)[y.astype(int)] # 2列に変換

## データを訓練データと評価データとテストデータに分割

In [79]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    shuffle=True,
    random_state=0,
    stratify=y
)

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train,
    y_train,
    test_size=0.2,
    shuffle=True,
    random_state=0,
    stratify=y_train
)

print("X_train", X_train.shape)
print("y_train", y_train.shape)
print("X_valid", X_valid.shape)
print("y_valid", y_valid.shape)
print("X_test", X_test.shape)
print("y_test", y_test.shape)

X_train (3913, 64, 64, 3)
y_train (3913, 2)
X_valid (979, 64, 64, 3)
y_valid (979, 2)
X_test (1223, 64, 64, 3)
y_test (1223, 2)


## 学習データの水増し

In [80]:
X_train_extend = []
y_train_extend = []

for X_, y_, j in zip(X_train, y_train, range(len(X_train))):
    print("X_train[", j, "]")
    img = Image.fromarray(X_) # numpy配列から画像に戻す。
    img_mirror = img.transpose(Image.FLIP_LEFT_RIGHT) # 左右反転
    
    # 回転
    for img_tmp in [img, img_mirror]:
    #for img_tmp in [img]:
        for i in range(0, 360, 15):
            #print("角度", i, "°")
            img_tmp2 = img_tmp.rotate(i)
            #print(img_tmp.shape)
            #print(X_train.shape)
            X_train_extend.append(np.array(img_tmp2))
            y_train_extend.append(y_)
        
            # フィルター
            img_tmp3 = img_tmp2.filter(ImageFilter.FIND_EDGES)
            X_train_extend.append(np.array(img_tmp3))
            y_train_extend.append(y_)
            img_tmp3 = img_tmp2.filter(ImageFilter.EDGE_ENHANCE)
            X_train_extend.append(np.array(img_tmp3))
            y_train_extend.append(y_)
            img_tmp3 = img_tmp2.filter(ImageFilter.EDGE_ENHANCE_MORE)
            X_train_extend.append(np.array(img_tmp3))
            y_train_extend.append(y_)
            img_tmp3 = img_tmp2.filter(ImageFilter.UnsharpMask(radius=5, percent=150, threshold=2))
            X_train_extend.append(np.array(img_tmp3))
            y_train_extend.append(y_)
            img_tmp3 = img_tmp2.filter(ImageFilter.UnsharpMask(radius=10, percent=200, threshold=5))
            X_train_extend.append(np.array(img_tmp3))
            y_train_extend.append(y_)

X_train[ 0 ]
X_train[ 1 ]
X_train[ 2 ]
X_train[ 3 ]
X_train[ 4 ]
X_train[ 5 ]
X_train[ 6 ]
X_train[ 7 ]
X_train[ 8 ]
X_train[ 9 ]
X_train[ 10 ]
X_train[ 11 ]
X_train[ 12 ]
X_train[ 13 ]
X_train[ 14 ]
X_train[ 15 ]
X_train[ 16 ]
X_train[ 17 ]
X_train[ 18 ]
X_train[ 19 ]
X_train[ 20 ]
X_train[ 21 ]
X_train[ 22 ]
X_train[ 23 ]
X_train[ 24 ]
X_train[ 25 ]
X_train[ 26 ]
X_train[ 27 ]
X_train[ 28 ]
X_train[ 29 ]
X_train[ 30 ]
X_train[ 31 ]
X_train[ 32 ]
X_train[ 33 ]
X_train[ 34 ]
X_train[ 35 ]
X_train[ 36 ]
X_train[ 37 ]
X_train[ 38 ]
X_train[ 39 ]
X_train[ 40 ]
X_train[ 41 ]
X_train[ 42 ]
X_train[ 43 ]
X_train[ 44 ]
X_train[ 45 ]
X_train[ 46 ]
X_train[ 47 ]
X_train[ 48 ]
X_train[ 49 ]
X_train[ 50 ]
X_train[ 51 ]
X_train[ 52 ]
X_train[ 53 ]
X_train[ 54 ]
X_train[ 55 ]
X_train[ 56 ]
X_train[ 57 ]
X_train[ 58 ]
X_train[ 59 ]
X_train[ 60 ]
X_train[ 61 ]
X_train[ 62 ]
X_train[ 63 ]
X_train[ 64 ]
X_train[ 65 ]
X_train[ 66 ]
X_train[ 67 ]
X_train[ 68 ]
X_train[ 69 ]
X_train[ 70 ]
X_train[ 71 ]
X_

X_train[ 556 ]
X_train[ 557 ]
X_train[ 558 ]
X_train[ 559 ]
X_train[ 560 ]
X_train[ 561 ]
X_train[ 562 ]
X_train[ 563 ]
X_train[ 564 ]
X_train[ 565 ]
X_train[ 566 ]
X_train[ 567 ]
X_train[ 568 ]
X_train[ 569 ]
X_train[ 570 ]
X_train[ 571 ]
X_train[ 572 ]
X_train[ 573 ]
X_train[ 574 ]
X_train[ 575 ]
X_train[ 576 ]
X_train[ 577 ]
X_train[ 578 ]
X_train[ 579 ]
X_train[ 580 ]
X_train[ 581 ]
X_train[ 582 ]
X_train[ 583 ]
X_train[ 584 ]
X_train[ 585 ]
X_train[ 586 ]
X_train[ 587 ]
X_train[ 588 ]
X_train[ 589 ]
X_train[ 590 ]
X_train[ 591 ]
X_train[ 592 ]
X_train[ 593 ]
X_train[ 594 ]
X_train[ 595 ]
X_train[ 596 ]
X_train[ 597 ]
X_train[ 598 ]
X_train[ 599 ]
X_train[ 600 ]
X_train[ 601 ]
X_train[ 602 ]
X_train[ 603 ]
X_train[ 604 ]
X_train[ 605 ]
X_train[ 606 ]
X_train[ 607 ]
X_train[ 608 ]
X_train[ 609 ]
X_train[ 610 ]
X_train[ 611 ]
X_train[ 612 ]
X_train[ 613 ]
X_train[ 614 ]
X_train[ 615 ]
X_train[ 616 ]
X_train[ 617 ]
X_train[ 618 ]
X_train[ 619 ]
X_train[ 620 ]
X_train[ 621 ]
X_train[ 6

X_train[ 1096 ]
X_train[ 1097 ]
X_train[ 1098 ]
X_train[ 1099 ]
X_train[ 1100 ]
X_train[ 1101 ]
X_train[ 1102 ]
X_train[ 1103 ]
X_train[ 1104 ]
X_train[ 1105 ]
X_train[ 1106 ]
X_train[ 1107 ]
X_train[ 1108 ]
X_train[ 1109 ]
X_train[ 1110 ]
X_train[ 1111 ]
X_train[ 1112 ]
X_train[ 1113 ]
X_train[ 1114 ]
X_train[ 1115 ]
X_train[ 1116 ]
X_train[ 1117 ]
X_train[ 1118 ]
X_train[ 1119 ]
X_train[ 1120 ]
X_train[ 1121 ]
X_train[ 1122 ]
X_train[ 1123 ]
X_train[ 1124 ]
X_train[ 1125 ]
X_train[ 1126 ]
X_train[ 1127 ]
X_train[ 1128 ]
X_train[ 1129 ]
X_train[ 1130 ]
X_train[ 1131 ]
X_train[ 1132 ]
X_train[ 1133 ]
X_train[ 1134 ]
X_train[ 1135 ]
X_train[ 1136 ]
X_train[ 1137 ]
X_train[ 1138 ]
X_train[ 1139 ]
X_train[ 1140 ]
X_train[ 1141 ]
X_train[ 1142 ]
X_train[ 1143 ]
X_train[ 1144 ]
X_train[ 1145 ]
X_train[ 1146 ]
X_train[ 1147 ]
X_train[ 1148 ]
X_train[ 1149 ]
X_train[ 1150 ]
X_train[ 1151 ]
X_train[ 1152 ]
X_train[ 1153 ]
X_train[ 1154 ]
X_train[ 1155 ]
X_train[ 1156 ]
X_train[ 1157 ]
X_train[

X_train[ 1613 ]
X_train[ 1614 ]
X_train[ 1615 ]
X_train[ 1616 ]
X_train[ 1617 ]
X_train[ 1618 ]
X_train[ 1619 ]
X_train[ 1620 ]
X_train[ 1621 ]
X_train[ 1622 ]
X_train[ 1623 ]
X_train[ 1624 ]
X_train[ 1625 ]
X_train[ 1626 ]
X_train[ 1627 ]
X_train[ 1628 ]
X_train[ 1629 ]
X_train[ 1630 ]
X_train[ 1631 ]
X_train[ 1632 ]
X_train[ 1633 ]
X_train[ 1634 ]
X_train[ 1635 ]
X_train[ 1636 ]
X_train[ 1637 ]
X_train[ 1638 ]
X_train[ 1639 ]
X_train[ 1640 ]
X_train[ 1641 ]
X_train[ 1642 ]
X_train[ 1643 ]
X_train[ 1644 ]
X_train[ 1645 ]
X_train[ 1646 ]
X_train[ 1647 ]
X_train[ 1648 ]
X_train[ 1649 ]
X_train[ 1650 ]
X_train[ 1651 ]
X_train[ 1652 ]
X_train[ 1653 ]
X_train[ 1654 ]
X_train[ 1655 ]
X_train[ 1656 ]
X_train[ 1657 ]
X_train[ 1658 ]
X_train[ 1659 ]
X_train[ 1660 ]
X_train[ 1661 ]
X_train[ 1662 ]
X_train[ 1663 ]
X_train[ 1664 ]
X_train[ 1665 ]
X_train[ 1666 ]
X_train[ 1667 ]
X_train[ 1668 ]
X_train[ 1669 ]
X_train[ 1670 ]
X_train[ 1671 ]
X_train[ 1672 ]
X_train[ 1673 ]
X_train[ 1674 ]
X_train[

X_train[ 2126 ]
X_train[ 2127 ]
X_train[ 2128 ]
X_train[ 2129 ]
X_train[ 2130 ]
X_train[ 2131 ]
X_train[ 2132 ]
X_train[ 2133 ]
X_train[ 2134 ]
X_train[ 2135 ]
X_train[ 2136 ]
X_train[ 2137 ]
X_train[ 2138 ]
X_train[ 2139 ]
X_train[ 2140 ]
X_train[ 2141 ]
X_train[ 2142 ]
X_train[ 2143 ]
X_train[ 2144 ]
X_train[ 2145 ]
X_train[ 2146 ]
X_train[ 2147 ]
X_train[ 2148 ]
X_train[ 2149 ]
X_train[ 2150 ]
X_train[ 2151 ]
X_train[ 2152 ]
X_train[ 2153 ]
X_train[ 2154 ]
X_train[ 2155 ]
X_train[ 2156 ]
X_train[ 2157 ]
X_train[ 2158 ]
X_train[ 2159 ]
X_train[ 2160 ]
X_train[ 2161 ]
X_train[ 2162 ]
X_train[ 2163 ]
X_train[ 2164 ]
X_train[ 2165 ]
X_train[ 2166 ]
X_train[ 2167 ]
X_train[ 2168 ]
X_train[ 2169 ]
X_train[ 2170 ]
X_train[ 2171 ]
X_train[ 2172 ]
X_train[ 2173 ]
X_train[ 2174 ]
X_train[ 2175 ]
X_train[ 2176 ]
X_train[ 2177 ]
X_train[ 2178 ]
X_train[ 2179 ]
X_train[ 2180 ]
X_train[ 2181 ]
X_train[ 2182 ]
X_train[ 2183 ]
X_train[ 2184 ]
X_train[ 2185 ]
X_train[ 2186 ]
X_train[ 2187 ]
X_train[

X_train[ 2643 ]
X_train[ 2644 ]
X_train[ 2645 ]
X_train[ 2646 ]
X_train[ 2647 ]
X_train[ 2648 ]
X_train[ 2649 ]
X_train[ 2650 ]
X_train[ 2651 ]
X_train[ 2652 ]
X_train[ 2653 ]
X_train[ 2654 ]
X_train[ 2655 ]
X_train[ 2656 ]
X_train[ 2657 ]
X_train[ 2658 ]
X_train[ 2659 ]
X_train[ 2660 ]
X_train[ 2661 ]
X_train[ 2662 ]
X_train[ 2663 ]
X_train[ 2664 ]
X_train[ 2665 ]
X_train[ 2666 ]
X_train[ 2667 ]
X_train[ 2668 ]
X_train[ 2669 ]
X_train[ 2670 ]
X_train[ 2671 ]
X_train[ 2672 ]
X_train[ 2673 ]
X_train[ 2674 ]
X_train[ 2675 ]
X_train[ 2676 ]
X_train[ 2677 ]
X_train[ 2678 ]
X_train[ 2679 ]
X_train[ 2680 ]
X_train[ 2681 ]
X_train[ 2682 ]
X_train[ 2683 ]
X_train[ 2684 ]
X_train[ 2685 ]
X_train[ 2686 ]
X_train[ 2687 ]
X_train[ 2688 ]
X_train[ 2689 ]
X_train[ 2690 ]
X_train[ 2691 ]
X_train[ 2692 ]
X_train[ 2693 ]
X_train[ 2694 ]
X_train[ 2695 ]
X_train[ 2696 ]
X_train[ 2697 ]
X_train[ 2698 ]
X_train[ 2699 ]
X_train[ 2700 ]
X_train[ 2701 ]
X_train[ 2702 ]
X_train[ 2703 ]
X_train[ 2704 ]
X_train[

X_train[ 3159 ]
X_train[ 3160 ]
X_train[ 3161 ]
X_train[ 3162 ]
X_train[ 3163 ]
X_train[ 3164 ]
X_train[ 3165 ]
X_train[ 3166 ]
X_train[ 3167 ]
X_train[ 3168 ]
X_train[ 3169 ]
X_train[ 3170 ]
X_train[ 3171 ]
X_train[ 3172 ]
X_train[ 3173 ]
X_train[ 3174 ]
X_train[ 3175 ]
X_train[ 3176 ]
X_train[ 3177 ]
X_train[ 3178 ]
X_train[ 3179 ]
X_train[ 3180 ]
X_train[ 3181 ]
X_train[ 3182 ]
X_train[ 3183 ]
X_train[ 3184 ]
X_train[ 3185 ]
X_train[ 3186 ]
X_train[ 3187 ]
X_train[ 3188 ]
X_train[ 3189 ]
X_train[ 3190 ]
X_train[ 3191 ]
X_train[ 3192 ]
X_train[ 3193 ]
X_train[ 3194 ]
X_train[ 3195 ]
X_train[ 3196 ]
X_train[ 3197 ]
X_train[ 3198 ]
X_train[ 3199 ]
X_train[ 3200 ]
X_train[ 3201 ]
X_train[ 3202 ]
X_train[ 3203 ]
X_train[ 3204 ]
X_train[ 3205 ]
X_train[ 3206 ]
X_train[ 3207 ]
X_train[ 3208 ]
X_train[ 3209 ]
X_train[ 3210 ]
X_train[ 3211 ]
X_train[ 3212 ]
X_train[ 3213 ]
X_train[ 3214 ]
X_train[ 3215 ]
X_train[ 3216 ]
X_train[ 3217 ]
X_train[ 3218 ]
X_train[ 3219 ]
X_train[ 3220 ]
X_train[

X_train[ 3674 ]
X_train[ 3675 ]
X_train[ 3676 ]
X_train[ 3677 ]
X_train[ 3678 ]
X_train[ 3679 ]
X_train[ 3680 ]
X_train[ 3681 ]
X_train[ 3682 ]
X_train[ 3683 ]
X_train[ 3684 ]
X_train[ 3685 ]
X_train[ 3686 ]
X_train[ 3687 ]
X_train[ 3688 ]
X_train[ 3689 ]
X_train[ 3690 ]
X_train[ 3691 ]
X_train[ 3692 ]
X_train[ 3693 ]
X_train[ 3694 ]
X_train[ 3695 ]
X_train[ 3696 ]
X_train[ 3697 ]
X_train[ 3698 ]
X_train[ 3699 ]
X_train[ 3700 ]
X_train[ 3701 ]
X_train[ 3702 ]
X_train[ 3703 ]
X_train[ 3704 ]
X_train[ 3705 ]
X_train[ 3706 ]
X_train[ 3707 ]
X_train[ 3708 ]
X_train[ 3709 ]
X_train[ 3710 ]
X_train[ 3711 ]
X_train[ 3712 ]
X_train[ 3713 ]
X_train[ 3714 ]
X_train[ 3715 ]
X_train[ 3716 ]
X_train[ 3717 ]
X_train[ 3718 ]
X_train[ 3719 ]
X_train[ 3720 ]
X_train[ 3721 ]
X_train[ 3722 ]
X_train[ 3723 ]
X_train[ 3724 ]
X_train[ 3725 ]
X_train[ 3726 ]
X_train[ 3727 ]
X_train[ 3728 ]
X_train[ 3729 ]
X_train[ 3730 ]
X_train[ 3731 ]
X_train[ 3732 ]
X_train[ 3733 ]
X_train[ 3734 ]
X_train[ 3735 ]
X_train[

In [81]:
# numpy配列に変換
X_train_extend = np.array(X_train_extend)
y_train_extend = np.array(y_train_extend)

print("X_train_extend", X_train_extend.shape)
print("y_train_extend", y_train_extend.shape)
print("X_valid", X_valid.shape)
print("y_valid", y_valid.shape)
print("X_test", X_test.shape)
print("y_test", y_test.shape)

X_train_extend (1126944, 64, 64, 3)
y_train_extend (1126944, 2)
X_valid (979, 64, 64, 3)
y_valid (979, 2)
X_test (1223, 64, 64, 3)
y_test (1223, 2)


## モデル構築

In [82]:
def create_model(MODEL, IMG_WIDTH, IMG_HEIGHT, IMG_CHANNELS):
    model = Sequential()
    
    if MODEL == 'CNN':    
        # 畳み込み層
        model.add(Conv2D(
            32, (3, 3), # ?
            padding='same', # ?
            activation='relu', # 活性化関数
            input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))) # 画像のサイズ
        
        # 畳み込み層
        model.add(Conv2D(
            32, (3, 3), # ?
            padding='same', # ?
            activation='relu', # 活性化関数
            input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))) # 画像のサイズ
    
        # プーリング層（情報量の削減）
        model.add(MaxPooling2D(pool_size=(2, 2)))
        
        # ドロップアウト層
        model.add(Dropout(0.25))
        
        # 畳み込み層
        model.add(Conv2D(
            64, (3, 3),
            padding='same',
            activation='relu'))
        
        # 畳み込み層
        model.add(Conv2D(
            64, (3, 3),
            padding='same',
            activation='relu'))
        
        # プーリング層
        model.add(MaxPooling2D(pool_size=(2, 2)))
        
        # 多層パーセプトロンに入力するにあたり４次元配列を１次元配列に変換
        model.add(Flatten())
        
        # 全結合層
        model.add(Dense(512, activation='relu'))
        
        # ドロップアウト層
        model.add(Dropout(0.5))
        
        # 全結合層
        model.add(Dense(2))
        
        # 活性化層
        model.add(Activation('softmax'))
        #model.add(Activation('relu'))
        
        # コンパイル
        model.compile(
            loss='categorical_crossentropy',
            optimizer='Adam',
            metrics=['accuracy'])
        
    elif MODEL == 'MyVGG16':
        # 畳み込み層
        model.add(Conv2D(
            16, (3, 3), # ?
            padding='same', # ?
            activation='relu', # 活性化関数
            input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))) # 画像のサイズ
        
        # 畳み込み層
        model.add(Conv2D(
            16, (3, 3), # ?
            padding='same', # ?
            activation='relu', # 活性化関数
            input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))) # 画像のサイズ
    
        # プーリング層（情報量の削減）
        model.add(MaxPooling2D(pool_size=(2, 2)))
        
        # ドロップアウト層
        #model.add(Dropout(0.25))
        
        """
        # 畳み込み層
        model.add(Conv2D(
            32, (3, 3),
            padding='same',
            activation='relu'))
        
        # 畳み込み層
        model.add(Conv2D(
            32, (3, 3),
            padding='same',
            activation='relu'))
        
        # プーリング層
        model.add(MaxPooling2D(pool_size=(2, 2)))
        
        # ドロップアウト層
        model.add(Dropout(0.25))
        
        # 畳み込み層
        model.add(Conv2D(
            64, (3, 3),
            padding='same',
            activation='relu'))
        
        # 畳み込み層
        model.add(Conv2D(
            64, (3, 3),
            padding='same',
            activation='relu'))
        
        # プーリング層
        model.add(MaxPooling2D(pool_size=(2, 2)))
        
        # ドロップアウト層
        model.add(Dropout(0.25))
        
        # 畳み込み層
        model.add(Conv2D(
            128, (3, 3),
            padding='same',
            activation='relu'))
        
        # 畳み込み層
        model.add(Conv2D(
            128, (3, 3),
            padding='same',
            activation='relu'))
        
        # プーリング層
        model.add(MaxPooling2D(pool_size=(2, 2)))
        
        # ドロップアウト層
        model.add(Dropout(0.25))
        
        # 畳み込み層
        model.add(Conv2D(
            256, (3, 3),
            padding='same',
            activation='relu'))
        
        # 畳み込み層
        model.add(Conv2D(
            256, (3, 3),
            padding='same',
            activation='relu'))
        
        # プーリング層
        model.add(MaxPooling2D(pool_size=(2, 2)))
        
        # ドロップアウト層
        model.add(Dropout(0.25))
        
        # 畳み込み層
        model.add(Conv2D(
            512, (3, 3),
            padding='same',
            activation='relu'))
        
        # 畳み込み層
        model.add(Conv2D(
            512, (3, 3),
            padding='same',
            activation='relu'))
        
        # プーリング層
        model.add(MaxPooling2D(pool_size=(2, 2)))
        
        # ドロップアウト層
        model.add(Dropout(0.25))
        """
        
        # 多層パーセプトロンに入力するにあたり４次元配列を１次元配列に変換
        model.add(Flatten())
        
        # 全結合層
        model.add(Dense(512, activation='relu'))
        
        # ドロップアウト層
        model.add(Dropout(0.5))
        
        # 全結合層
        model.add(Dense(1))
        
        # 活性化層
        #model.add(Activation('softmax'))
        model.add(Activation('relu'))
        
        # コンパイル
        model.compile(
            loss='mse',
            optimizer='Adam',
            metrics=['mae'])
        
    elif MODEL == 'VGG16':
        # VGG16
        model = VGG16(
            include_top=True,
            weights=None,
            input_tensor=None,
            input_shape=(IMG_WIDTH, IMG_HEIGHT, IMG_CHANNELS),
            #pooling=None,
            classes=1,
        )
        
        
        # 全結合層
        #model.add(Dense(NUM_CLASSES))
        
        # 活性化層
        #model.add(Activation('softmax'))
        
        # コンパイル
        model.compile(
            loss='mse',
            optimizer='Adam',
            metrics=['mae'])
        
    elif MODEL == 'VGG19':
        model = VGG19(
            include_top=True,
            weights=None,
            input_tensor=None,
            input_shape=(IMG_WIDTH, IMG_HEIGHT, IMG_CHANNELS),
            #pooling=None,
            classes=1
        )
        
        
        # コンパイル
        model.compile(
            loss='mse',
            optimizer='Adam',
            metrics=['mae'])
        
    elif MODEL == 'ResNet50':
        model = ResNet50(
            include_top=True,
            weights=None,
            input_tensor=None,
            input_shape=(IMG_WIDTH, IMG_HEIGHT, IMG_CHANNELS),
            #pooling=None,
            classes=1
        )
        
        # コンパイル
        model.compile(
            loss='mse',
            optimizer='Adam',
            metrics=['mae'])
        
    elif MODEL == 'InceptionResNetV2':
        model = InceptionResNetV2(
            include_top=True,
            weights=None,
            input_tensor=None,
            input_shape=(IMG_WIDTH, IMG_HEIGHT, IMG_CHANNELS),
            #pooling=None,
            classes=2
        )
        
        # コンパイル
        model.compile(
            loss='mse',
            optimizer='Adam',
            metrics=['mae'])
        
    return model

In [83]:
BATCH_SIZE = 32
EPOCHS = 100
VERBOSE = 1
VALIDATION_SPLIT = 0.2
MODEL = 'CNN'#'ResNet50'#'CNN'

In [84]:
# モデルの作成
model = create_model(MODEL, img_size, img_size, 3)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 64, 64, 32)        9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 32, 32, 64)        18496     
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 32, 32, 64)        36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 16, 64)        0         
__________

In [85]:
early_stopping = EarlyStopping(patience=0, verbose=1)

# 学習（戻り値はログ）
hist = model.fit(
    X_train,
    y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks = [early_stopping],
    verbose=VERBOSE, # 出力の詳細
    validation_data=(X_valid, y_valid)#,
    #shuffle = True,
    #validation_split=VALIDATION_SPLIT
)

Train on 3913 samples, validate on 979 samples
Epoch 1/100
3913/3913 [==============================] - 4s 896us/step - loss: 1.9517 - acc: 0.8781 - val_loss: 1.9263 - val_acc: 0.8805
Epoch 2/100
3913/3913 [==============================] - 3s 642us/step - loss: 1.9236 - acc: 0.8807 - val_loss: 1.9263 - val_acc: 0.8805
Epoch 00002: early stopping


In [86]:
score = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE, verbose=1)
print("損失関数:", score[0])
print("評価関数:", score[1])

1223/1223 [==============================] - 0s 236us/step
損失関数: 1.9241553440702224
評価関数: 0.8806214227797258


In [87]:
X_test.shape

(1223, 64, 64, 3)

In [88]:
pred = model.predict(X_test)#.flatten()
#df_pred = pd.DataFrame([pred, y_test])
#df_pred = df_pred.T

## 画像を分類して保存

In [89]:
"""
for x, y, p, i in zip(X_test, y_test, pred, range(len(pred))):
    #print(int(y[0]))
    #if int(int(y[0])) == 0:
    img = Image.fromarray(x)
    img_url = 'scraped_img/test_division/' + str(int(y[1])) + '/' + str(i) + '.jpg'
    #print(img_url)
    img.save(img_url, quality=100) # 保存
    
#print(pred.shape)
#print(Y_test.shape)
#print(pred)
#print(Y_test)
"""

"\nfor x, y, p, i in zip(X_test, y_test, pred, range(len(pred))):\n    #print(int(y[0]))\n    #if int(int(y[0])) == 0:\n    img = Image.fromarray(x)\n    img_url = 'scraped_img/test_division/' + str(int(y[1])) + '/' + str(i) + '.jpg'\n    #print(img_url)\n    img.save(img_url, quality=100) # 保存\n    \n#print(pred.shape)\n#print(Y_test.shape)\n#print(pred)\n#print(Y_test)\n"

In [90]:
#df_pred

## 実際に分類したいデータを分類する

In [91]:
X_target = []

#img_path_list = glob.glob('image/ddh-01_ddh-10/*')
#img_path_list = glob.glob('image/ddh-01_under_100000/*') # 平均絶対誤差17000
#img_path_list = glob.glob('image/ddh-01_ddh-10_under_100000/*') # 平均絶対誤差16000
#img_path_list = glob.glob('image/ddh-01_ddh-10_under_50000/*') # 平均絶対誤差10000
img_path_list = glob.glob('image/original/ddh-01/*') # 平均絶対誤差9000
#img_path_list = glob.glob('image/ddh-01_ddh-10_13000_41000/*') # 平均絶対誤差7000

for path in img_path_list:
    print(path)
    img = Image.open(path) # 開いて
    img = expand2square(img, (255, 255, 255)) # 正方形に整形して
    img = img.resize((img_size, img_size)) # サイズを調整する
    
    X_target.append(np.array(img)) # 学習データ

image/original/ddh-01\00001000_94e2613d-c6ad-4313-b19a-c11c516ea72f.jpeg
image/original/ddh-01\00001000_bb5d7e32-bede-47f1-802e-45ec08d34e75.jpeg
image/original/ddh-01\00001800_6db1651f-1ced-4c84-b40d-ef86f77a62b9.jpeg
image/original/ddh-01\00002500_2cf026cc-b16b-4346-af40-31f94573c1aa.jpeg
image/original/ddh-01\00002500_3b856c25-d340-49c3-9540-f2330448c5d9.jpeg
image/original/ddh-01\00002500_565b1d95-e23d-45f9-8144-38592ff0ab06.jpeg
image/original/ddh-01\00002500_9b90ed6c-d4e2-4385-91b4-8b7231fb1334.jpeg
image/original/ddh-01\00002500_bfddde13-779f-496d-8a36-1a525b5b1b70.jpeg
image/original/ddh-01\00002600_5f920652-eed0-404d-9a63-f7e8f57dd6b2.jpeg
image/original/ddh-01\00003300_e15f0224-0069-4058-954e-e7d6833494ce.jpeg
image/original/ddh-01\00003500_01eba574-8083-45e0-bb6a-8709995e95f7.jpeg
image/original/ddh-01\00003600_59215fbc-744b-4c5b-b1be-7ffc77cab60e.jpeg
image/original/ddh-01\00003800_41a2f590-035d-4363-b7c1-8803903ed6a1.jpeg
image/original/ddh-01\00003800_7ff0db8e-d12c-4219-9

image/original/ddh-01\00009500_bf373e00-8027-4cbd-a479-9e507f146d94.jpeg
image/original/ddh-01\00009550_25c92c70-5977-4ef8-9cd1-3d5056a16c2b.jpeg
image/original/ddh-01\00009750_1a9e3f11-5b34-42ad-bdf0-e2030f725c30.jpeg
image/original/ddh-01\00009750_1fc320ac-8d68-49f1-93fc-07691956b941.jpeg
image/original/ddh-01\00009750_7a3431a3-f562-4897-8993-94de17d4afa7.jpeg
image/original/ddh-01\00009750_7dc36502-a3df-42ed-ac61-72e859e00dbf.jpeg
image/original/ddh-01\00009750_c17de06b-8db6-4271-8abe-d38e08371c43.jpeg
image/original/ddh-01\00009999_7a8d6bbe-ec64-45c5-b6a6-883d9f44866e.jpeg
image/original/ddh-01\00010000_0761608b-852e-4a9f-8182-251370aaf9ec.jpeg
image/original/ddh-01\00010000_1265fb00-e461-4533-aa5c-e397483f4b99.jpeg
image/original/ddh-01\00010000_2476ecb9-00d7-4274-a6b7-54f7b0ee7bb7.jpeg
image/original/ddh-01\00010000_4a87e1e6-d15b-4807-b6a3-6ea5118f340e.jpeg
image/original/ddh-01\00010000_4fe99092-4365-411a-b785-49ae63b8f3cf.jpeg
image/original/ddh-01\00010000_55e53e24-7fd5-4bb7-9

image/original/ddh-01\00015500_28520d98-f3bb-4add-9dce-6c90f3a382b0.jpeg
image/original/ddh-01\00015500_2b4a6278-57ee-4ce9-851c-2ae5bb6beaf1.jpeg
image/original/ddh-01\00015500_4d73572c-d47e-45fe-86d3-b175625197d4.jpeg
image/original/ddh-01\00015500_505c433d-10fb-45e1-98de-2e81fe333120.jpeg
image/original/ddh-01\00015500_69a02a13-4272-45af-aab1-38511802b05c.jpeg
image/original/ddh-01\00015500_72f2f5e9-7708-4bca-8ba4-e2bc99bd8354.jpeg
image/original/ddh-01\00015500_8876a2b5-6f94-486f-a06e-de79e6488bba.jpeg
image/original/ddh-01\00015500_92e9a374-f43c-4d77-a2cc-eb41ca3f1f43.jpeg
image/original/ddh-01\00015500_aae98301-6b00-408a-8742-cbfc88e4d102.jpeg
image/original/ddh-01\00015500_b4ca36fd-72f8-44f3-baad-0eb5b8e54e21.jpeg
image/original/ddh-01\00015500_bd95de3d-0470-464b-b2fe-114850b68fce.jpeg
image/original/ddh-01\00015500_ca6adaac-8e70-426a-8f51-74d9554a80d2.jpeg
image/original/ddh-01\00015500_f5e0e6bc-634f-4460-b5bc-c7c57a3da58d.jpeg
image/original/ddh-01\00015500_fe9af30d-a2eb-4f64-8

image/original/ddh-01\00023000_2ad52e2c-3037-4a69-a5a5-c578950a26cf.jpeg
image/original/ddh-01\00023000_61b6f1ef-9c43-4910-8cfe-e9d88b401c96.jpeg
image/original/ddh-01\00023000_775dd96a-c879-4f0c-b148-e98251c79e9c.jpeg
image/original/ddh-01\00023000_816dbee5-218e-4f50-b55d-7f379cd9edbf.jpeg
image/original/ddh-01\00023000_c2a6b120-9e74-4ac0-b0e0-03b1bd16bc86.jpeg
image/original/ddh-01\00023000_df18854c-a295-47cc-a1ef-7865b975073c.jpeg
image/original/ddh-01\00023000_e3186851-df0c-4db8-8ebf-5954f21bf3ea.jpeg
image/original/ddh-01\00023500_6f57b9dd-589c-4a91-8aab-675db8317330.jpeg
image/original/ddh-01\00023500_86bdf329-0917-4e7f-889f-2c6c31920c4e.jpeg
image/original/ddh-01\00023500_bf814ca4-0829-4f0f-9314-3930fe19eaeb.jpeg
image/original/ddh-01\00024000_17c7cea2-e58a-4c63-b994-334e6392764c.jpeg
image/original/ddh-01\00024000_289e1b3f-0fca-4448-bb66-e0b7f70d0c84.jpeg
image/original/ddh-01\00024000_37934297-e460-48e6-94b1-dbf25bf2677c.jpeg
image/original/ddh-01\00024000_3acd0b67-a524-4639-b

image/original/ddh-01\00031000_526abbb3-a6c9-448c-a11b-d0a0a37cf32b.jpeg
image/original/ddh-01\00031000_7c8917c2-9429-4043-aa31-afb2ff543af9.jpeg
image/original/ddh-01\00031000_93a6435d-a106-44b4-ad33-57b854cf8e8b.jpeg
image/original/ddh-01\00031000_b0b2218a-f6c1-4b4b-bd3f-b45c1cb5949e.jpeg
image/original/ddh-01\00031000_cd73d93d-d21a-4225-93db-52f77f0b4b4a.jpeg
image/original/ddh-01\00031100_e32458ac-6e73-4132-95f5-d3760eba6715.jpeg
image/original/ddh-01\00031500_35bdb5d1-742d-467f-90a2-44fa2ae76488.jpeg
image/original/ddh-01\00031500_3a59c2f9-159f-4f12-970c-4bcffdef485a.jpeg
image/original/ddh-01\00031500_41186213-aab0-40db-8f62-94fbcadb0965.jpeg
image/original/ddh-01\00031500_af88334d-cd00-4c7c-8830-3fda0c8bc70f.jpeg
image/original/ddh-01\00031500_bb63526b-9870-4c59-a453-b4f938da2e2e.jpeg
image/original/ddh-01\00031500_cc07e067-0a02-4919-bb02-3541eff159ba.jpeg
image/original/ddh-01\00032000_42745d79-c4d7-4566-ace7-5f38862985e3.jpeg
image/original/ddh-01\00032000_969a7123-e1f6-4a73-a

image/original/ddh-01\00043500_21a0e0ad-e2b5-411d-9b55-5bab1608b99a.jpeg
image/original/ddh-01\00043500_4a0246ba-7d4a-4f2f-b8e9-8d0e69260246.jpeg
image/original/ddh-01\00043500_6798d6c8-baaf-4daa-8508-a0bf56ed8c92.jpeg
image/original/ddh-01\00043500_76ccc127-e1fe-44d1-9c45-f4414e012dc7.jpeg
image/original/ddh-01\00044000_176084b6-9652-4375-96fa-7c53ae9ebbc9.jpeg
image/original/ddh-01\00044000_5fd6954e-5243-4852-b071-cd84984df835.jpeg
image/original/ddh-01\00044000_8553ec1c-1fb3-41ba-8824-02cabbefe73d.jpeg
image/original/ddh-01\00044100_b7e8644b-1abc-43d2-baa0-35ed797b2ce9.jpeg
image/original/ddh-01\00044500_877172ca-b96a-4326-b397-85678991bbe4.jpeg
image/original/ddh-01\00044700_25331e88-2356-4554-b73a-5db2f57e2c01.jpeg
image/original/ddh-01\00045000_41a9c916-6108-4f14-a0b6-566cc4b1a39b.jpeg
image/original/ddh-01\00045400_25878d93-75ad-48af-922c-2e49bad4706a.jpeg
image/original/ddh-01\00045500_060329f7-29d9-42d6-af23-52339e395272.jpeg
image/original/ddh-01\00045500_088f43a6-2e2a-4bbc-a

image/original/ddh-01\00072001_98d26d88-706a-485f-82d4-c7210d6506fa.jpeg
image/original/ddh-01\00073000_25862583-6fd7-4788-9ab8-778099a93ddc.jpeg
image/original/ddh-01\00073000_9535f8f2-9948-4faf-b1da-856453f3aa9a.jpeg
image/original/ddh-01\00075000_a8e05537-240a-437c-93dc-93d67ec10485.jpeg
image/original/ddh-01\00075000_de481fa7-5228-48c0-a360-fde73a5ef8fd.jpeg
image/original/ddh-01\00076000_82f678ca-fff3-478f-bb9b-83c7f305080a.jpeg
image/original/ddh-01\00076000_9d06c2fe-7569-4348-b3e4-4aa7327c5060.jpeg
image/original/ddh-01\00076000_ef97b0a4-38dc-4071-8fec-41099c14a6ec.jpeg
image/original/ddh-01\00077000_fe46b842-f714-44a8-992e-3ce7b275f7f5.jpeg
image/original/ddh-01\00077050_57bd4ba8-9d5f-4d6f-8bc3-fc9c8070f7e6.jpeg
image/original/ddh-01\00078000_9a5cac01-a925-497f-bba2-1252e8f79519.jpeg
image/original/ddh-01\00078000_e59dfc5f-0b08-4ea8-8d88-1949acf25a53.jpeg
image/original/ddh-01\00079000_38368348-ec75-4d70-89b6-0d999ad3099d.jpeg
image/original/ddh-01\00080000_50ca7d77-a8cb-4983-9

In [92]:
# numpy配列に変換
X_target = np.array(X_target)

In [93]:
pred = model.predict(X_target)#.flatten()
#df_pred = pd.DataFrame([pred, y_test])
#df_pred = df_pred.T

In [95]:
print(pred)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [94]:
for x, p, i in zip(X_target, pred, range(len(pred))):
    img = Image.fromarray(x)
    img_url = 'scraped_img/test_division/' + str(int(p[1])) + '/' + str(i) + '.jpg'
    img.save(img_url, quality=100) # 保存
    
#print(pred.shape)
#print(Y_test.shape)
#print(pred)
#print(Y_test)

## MSE   = Mean Squared Error        = 平均二乗誤差
# $\frac{1}{n}\sum_{i=1}^{n}(f_i-y_i)^2$

## RMSE = Root Mean Square Error = 平均平方二乗誤差
ルートの中身を二乗しているため、MAEと比較して外れ値をより大きな誤差として扱う傾向がある \
# $\sqrt{\frac{1}{n}\sum_{i=1}^{n}(f_i-y_i)^2}$

## MAE   = Mean Absolute Error        = 平均絶対誤差
# $\frac{1}{n}\sum_{i=1}^{n}|f_i-y_i|$

## RMSPE = Root Mean Square Percentage Error = 平均平方二乗誤差率
# $\sqrt{\frac{100}{n}\sum_{i=1}^{n}(\frac{f_i-y_i}{y_i})^2}$

## MAPE    = Mean Absolute Percentage Error = 平均絶対誤差率
# $\frac{100}{n}\sum_{i=1}^{n}|\frac{f_i-y_i}{y_i}|$

|指標名  |特徴　                        |損失関数  |最適化指標|評価関数|KPI(重要標石評価指標)|
|--------|------------------------------|----------|----------|--------|---------------------|
|RMSE    |大きいエラーを重要視する      |◯        |◯        |△      |X                    |
|RMSLE   |予測と実測の比率と下振れを重視|△        |◯        |△      |X                    |
|MAE     |誤差の幅を等しく扱う          |△        |◯        |◯      |△                   |
|MAPE    |誤差を百分率で考慮            |X         |△        |◯      |◯                   |
|Poisson |離散確率分布                  |△(BP依存)|◯        |△      |X                    |
|Gammma  |連続確率分布                  |△(BP依存)|◯        |△      |X                    |
|Tweedie |0か0以外か                    |△(BP依存)|◯        |△      |X                    |
|R Square|決定係数                      |X         |△        |△      |X                    |